In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
import matplotlib.pyplot as plt

In [ ]:
# dataframe
df = pd.read_csv('./data/boston.csv')
df.describe()

In [ ]:
df =df.values

In [ ]:
x_data = df[:,:12]
for i in range(12):
    x_data[:,i] = x_data[:,i] / (x_data[:,i].max() - x_data[:,i].min())
y_data = df[:,12].reshape(-1,1)

In [ ]:
x_data.shape, y_data.shape

In [ ]:
train_epochs = 500
learning_rate = 0.01

x = tf.placeholder(dtype=tf.float32, shape=(None, 12), name="X")
y = tf.placeholder(dtype=tf.float32, shape=(None,  1), name="Y")
w = tf.Variable(np.random.randn(12, 1), dtype=tf.float32, name="W")
b = tf.Variable(0., dtype=tf.float32, name="b")

with tf.name_scope("model"):
    def model(x, w, b):
        return tf.matmul(x, w) + b
    pred = model(x, w, b)
    
with tf.name_scope("loss"):
    loss_function = tf.reduce_mean(tf.square(y-pred))
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss_function)
sum_loss_op = tf.summary.scalar("loss", loss_function)
merged = tf.summary.merge_all()

In [ ]:
lost_list = []
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    writer = tf.summary.FileWriter("./log/", sess.graph)
    for epoch in range(50):
        loss_sum = 0.
        for xs, ys in zip(x_data, y_data):
            xs = np.reshape(xs, (-1, 12))
            ys = np.reshape(ys, (-1, 1))
            _, summ_str, loss = sess.run([optimizer, sum_loss_op, loss_function], feed_dict={x: xs, y: ys})
            writer.add_summary(summ_str, epoch)
            loss_sum += loss
        print(epoch, "loss: ", loss_sum / len(x_data))
        x_data = shuffle(x_data)
        y_data = shuffle(y_data)
        lost_list.append(loss_sum / len(x_data))
    
    print(epoch, "loss: ", loss_sum / len(x_data))
    n = 9
    x_test = x_data[n]
    x_test = np.reshape(x_test, (1, -1))
    predict = sess.run(pred, feed_dict={x: x_test})
    
    print(predict, "vs ", y_data[n])

In [ ]:
plt.plot(lost_list)